In [322]:
import psycopg2
from flask import Flask, session, request, redirect
from markupsafe import escape

app = Flask(__name__)
app.config['SECRET_KEY'] = 'shh_dont_tell'

In [323]:
#ESTABLISHES CONNECTION FUNCTION- to call
def establish_connection():
    return psycopg2.connect(
    host = "localhost",
    database= "Real_Estate_Managment",
    user = "postgres",
    password = "Nata1204",
    port = 5432)

In [324]:
#FUNCTIONS

#Get role

def get_role(User_id, cursor):
    cursor.execute("SELECT * FROM User_renter WHERE userid = %s;", (User_id,))
    if cursor.fetchone() is not None:
        return 'renter'
    else:
        cursor.execute("SELECT * FROM User_agent WHERE userid = %s;", (User_id,))
        if cursor.fetchone() is not None:
            return 'agent'
        else:
            return "Invalid"
    

In [325]:
@app.route("/")
def home():
    user_id = session.get('UserID')
    if user_id:
        return redirect(f'/dashboard/{user_id}')
    return redirect('/Login')

In [326]:
@app.route('/dashboard/<user_id>')
def dashboard(user_id):

    return f'''<!DOCTYPE html>
    <html>
    <head>
            <title> Dashboard! </title>
    </head>
    <body>
        <h1 style= "text-align:center"> Dashboard for user {escape(user_id)}! </h1>

    <form method = "GET">
                <a href = "/CC" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Manage Credit Cards </a>
                <a href = "/Properties"style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self">Search Properties</a>
                <a href = "/Bookings" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Manage Bookings</a>
                <a href = "/Login" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Login </a>
                <a href = "/register" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Register</a>
     </form>
    </body>
    </html>'''


In [327]:
@app.route("/Login", methods=['GET','POST'])
def Login():
    if request.method == 'POST':
        email_address = request.form['email_address']
        try:
            conn = establish_connection()

            cursor = conn.cursor()
            cursor.execute('SELECT UserID FROM "user" WHERE email_address = %s', (email_address,))
            result = cursor.fetchone()

            if result:
                session['UserID'] = str(result[0])
                return redirect('/')
            else:
                return redirect('/register')
        except Exception as e:
            return f"Login error: {e}"
        finally:
            if conn is not None:
                conn.close()

    return '''<!DOCTYPE html>
    <html>
    <body>

    <h2>Real Estate Login</h2>

    <form action="/Login" method = "POST">
    <label for="Email">Email Address:</label><br>
    <input type="text" id="Email" name="email_address"><br>
    <input type = "submit" value = "Login">
    </form>
    </body>
    </html>'''


In [328]:
def add_user(cursor,email_address,first_name,mid_name,last_name):
  try:
    cursor.execute('INSERT INTO "user" (email_address,first_name,mid_name,last_name) VALUES (%s,%s,%s,%s) RETURNING UserID;',(email_address,first_name,mid_name,last_name))
    user_id = cursor.fetchone()[0]
    return user_id
  except Exception as e:
    print("An ERROR has occurred:", e)
    return None

In [329]:
@app.route('/register', methods=['GET','POST'])
def register():
    conn = establish_connection()
    cursor = conn.cursor()

    if request.method == 'POST':
        try:
            email_address = request.form['email_address']
            first_name = request.form["first_name"]
            mid_name =request.form["mid_name"]
            last_name = request.form["last_name"]
            role = request.form['role'].lower()

            cursor.execute('INSERT INTO "user" (email_address,first_name,mid_name,last_name) VALUES (%s,%s,%s,%s) RETURNING UserID;',(email_address,first_name,mid_name,last_name))
            user_id = cursor.fetchone()[0]
            
            if role =='renter':
                moveindate = request.form['moveindate']
                location= request.form['location']
                budget = request.form['budget']

                cursor.execute("INSERT INTO User_renter (UserID, moveindate, location, budget) VALUES (%s,%s,%s,%s);",(user_id,moveindate,location, budget))          

            elif role =='agent':
                job_title = request.form['job_title']
                agency = request.form['agency']
                phone_num = request.form['phone_num']

                cursor.execute("INSERT INTO User_agent (UserID, job_title,agency, phone_num) VALUES (%s,%s,%s,%s);",(user_id,job_title,agency,phone_num))
            conn.commit()
            session['UserID'] = user_id
            return redirect('/')

        except Exception as e:
            return f"Registration error: {e}"
        finally:
            if conn is not None:
                conn.close()
        
    return '''
<!DOCTYPE html>
<html>
<body>

<h2>Real Estate Registration</h2>

<form method = "POST">
  <label for="Email">Email Address:</label><br>
  <input type="text" id="Email" name="email_address"><br>

  <label>First Name:</label><br>
  <input type="text" name="first_name"><br>

   <label> Middle Name:</label><br>
  <input type="text" name="mid_name"><br>

   <label>Last Name:</label><br>
  <input type="text" name="last_name"><br>

   <label>Role (renter or agent):</label><br>
  <input type="text" name="role"><br><br>

  <strong>Renter Info:</strong><br>

   <label>Move-In Date:</label><br>
  <input type="text" name="moveindate"><br>

   <label>Location:</label><br>
  <input type="text" name="location"><br>

   <label>Budget:</label><br>
  <input type="text" name="budget"><br><br>

  <strong>Agent Info:</strong><br>

   <label>Job Title:</label><br>
  <input type="text" name="job_title"><br>

   <label>Agency:</label><br>
  <input type="text" name="agency"><br>

   <label>Phone Number:</label><br>
  <input type="text" name="phone_num"><br><br>

  <input type="submit" value="Register">
</form> 
</body>
</html>'''

            

In [330]:
@app.route("/CC")
def CC():
    conn = establish_connection()
    cursor = conn.cursor()

    user_id = session.get('UserID')
    
    if not user_id:
        return redirect('/Login')
    
    cursor.execute("SELECT * FROM CC WHERE UserID = %s;",(str(user_id),))
    cards = cursor.fetchall()
    cursor.close()
    conn.close()

    rows = ""
    for card in cards:
        row = "<tr>" + "".join(f"<td>{value}</td>" for value in card) + "</tr>\n"
        rows+= row



    return f'''
    <!DOCTYPE html>
<html lang="en">
 
<head>
    <meta charset="UTF-8">
    <title> Credit Card System</title>

    <style>
        table {{margin: 0 auto, font-size: medium; border: 1px solid black;}}

        h1 {{text-align: center; color =black; font-size: large; font-family: 'Calibri' }}
        td {{background-color:#E4F5D4; border: 1px solid black;}}
        th,
        td{{font-weight: bold; border: 1px solid black; padding: 10px; text-align: center;}}

        td {{font-weight: lighter;}}
    </style>
</head>
<body>
    <h1> Your Credit Cards<h1>
    <form method = "GET">
                <a href = "/CC_Add" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Add A Credit Card </a>
                <a href = "/CC_Delete" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Delete A Credit Card </a>
                <a href = "/CC_Update" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Update A Credit Card </a>
    <section>
    <table>
        <tr>
            <th>CardID</th>
            <th>CC Number</th>
            <th>UserID</th>
            <th>House Number</th>
            <th>House APT</th>
            <th>Street</th>
            <th>City</th>
            <th>State</th>
            <th>ZIP</th>
            <th>Country</th>
            <th>Expiration Date</th>
            <th>Cardholder Name</th>
        </tr>
        {rows}
        </table>
        </section>
    </body>
    </html>

'''


    

In [331]:
@app.route("/CC_Add", methods=['GET','POST'])
def add_CC():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
            return redirect('/Login')
   
   try:
      if request.method == 'POST':  
         
         role = get_role(user_id,cursor)

         if role != 'renter':
            print("Access denied!")
            return redirect('/')
   
         CardID = request.form['CardID']
         CC_num = request.form['CC_num']
         bill_num=request.form['bill_num']
         bill_apt=request.form['bill_apt']
         bill_street=request.form['bill_street']
         bill_city=request.form['bill_city']
         bill_state=request.form['bill_state']
         bill_zip=request.form['bill_zip']
         bill_country=request.form['bill_country']
         Expiration_date=request.form['Expiration_date']
         Cardholder_Name=request.form['Cardholder_Name']

         cursor.execute("CALL add_CC(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);",(CardID,CC_num,str(user_id),bill_num,bill_apt,bill_street,bill_city,bill_state,bill_zip,
            bill_country,Expiration_date,Cardholder_Name))
         conn.commit() 
         return redirect('/CC')
   except Exception as e:
      return f"CC error: {e}"
   finally:
      if conn is not None:
         conn.close()

   return'''   <!DOCTYPE html>
<html lang="en">
<head>
    <h1> Add a Credit Card </h1>
</head>

<form method = "POST">
   CardID: <input type="text" id="CardID" name="CardID"><br>

  CC Number:
  <input type="text" name="CC_num"><br>

   Expiration Date:
  <input type="text" name="Expiration_date"><br>

   Cardholder Name:
  <input type="text" name="Cardholder_Name"><br>

   <strong>Card Address:</strong><br>

   House Number:
  <input type="text" name="bill_num"><br>

   APT:
  <input type="text" name="bill_apt"><br>

   Street:
  <input type="text" name="bill_street"><br>

   City:
  <input type="text" name="bill_city"><br>

   State:
  <input type="text" name="bill_state"><br>

   Zip Code:
  <input type="text" name="bill_zip"><br>

   Country:
  <input type="text" name="bill_country"><br>

  <input type="submit" value="Add">
</form> 
</body>
</html>
'''

In [332]:
@app.route("/CC_Delete", methods=['GET','POST'])
def delete_CC():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
            return redirect('/Login')
   
   try:
      if request.method == 'POST':  
         
         role = get_role(user_id,cursor)
         CC_num=request.form['CC_num']


         if role != 'renter':
            print("Access denied!")
            return redirect('/')
        
         cursor.execute("CALL delete_CC2(%s,%s);",(CC_num, str(user_id)))
         conn.commit()
         return redirect('/CC')
      
   except Exception as e:
      return f"CC error: {e}"
   finally:
      if conn is not None:
         conn.close()
   return '''   <!DOCTYPE html>
<html lang="en">
<body>
    <h1> Delete a Credit Card </h1>
</body>

<form method = "POST"> 

   CC Number:
  <input type="text" name="CC_num"><br>

  <input type="submit" value="Delete">
</form> 
</body>
</html>
'''

In [333]:
@app.route("/CC_Update", methods=['GET','POST'])
def update_CC():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
            return redirect('/Login')
   
   try:
      if request.method == 'POST':  
         
         role = get_role(user_id,cursor)
         if role != 'renter':
            print("Access denied!")
            return
         
         CardID = request.form['CardID']
         field= request.form['field']
         new_value= request.form['new_value']

         cursor.execute("SELECT * FROM CC WHERE CardID = %s AND UserID = %s;", (CardID, str(user_id)))
         if not cursor.fetchone():
            return redirect('/CC')
        
         fields_available = ['CC_num','bill_num','bill_apt','bill_street','bill_city','bill_state','bill_zip','bill_country','Expiration_date','Cardholder_Name']
         if field not in fields_available:
            print(f"Invalid field name: {field}")
            return redirect('/CC_Update')
        
         cursor.execute(f"UPDATE CC SET {field} = %s WHERE CardID = %s;",(new_value,CardID))
         conn.commit()
         return redirect('/CC')



   except Exception as e:
      return f"CC error: {e}"
   finally:
      if conn is not None:
         conn.close()

   return '''   <!DOCTYPE html>
<html lang="en">
<body>
    <h1> Update a Credit Card </h1>
</body>

<form method = "POST"> 

   CCID:
  <input type="text" name="CardID"><br>
   Field to change: [CC_num,bill_num,bill_apt,bill_street,bill_city,bill_state,bill_zip,bill_country,Expiration_date,Cardholder_Name]
  <input type="text" name="field"><br>
   New Value:
  <input type="text" name="new_value"><br>


  <input type="submit" value="Update">
</form> 
</body>
</html>
'''



In [334]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/May/2025 23:24:48] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [02/May/2025 23:24:48] "GET /dashboard/3 HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2025 23:24:50] "GET /CC HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2025 23:24:52] "GET /CC_Update HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2025 23:25:00] "GET /CC_Update HTTP/1.1" 200 -
[2025-05-02 23:25:16,543] ERROR in app: Exception on /CC_Update [POST]
Traceback (most recent call last):
  File "c:\Users\natib\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natib\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 920, in full_dispatch_request
    return self.finalize_request(rv)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natib\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py"

Access denied!


127.0.0.1 - - [02/May/2025 23:25:18] "GET /CC_Update HTTP/1.1" 200 -
[2025-05-02 23:25:24,205] ERROR in app: Exception on /CC_Update [POST]
Traceback (most recent call last):
  File "c:\Users\natib\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natib\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 920, in full_dispatch_request
    return self.finalize_request(rv)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natib\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 939, in finalize_request
    response = self.make_response(rv)
               ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natib\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1212, in make_response
    raise TypeError(
TypeError: The view function for 'update_CC' did not return a vali

Access denied!
